In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from HuCapa_trainer import evaluateFromList
from Ecapa_trainer import EcapaTrainer
from HuCapa_trainer import HuCapaTrainer
from neural_compressor.training import prepare_compression
from neural_compressor.config import DistillationConfig, KnowledgeDistillationLossConfig
from models.HuCapa import HuCapa
from models.CustomEcapa import ECAPA_TDNN
import torch
import time
from tuneThreshold import *

/home/iref/Repos/voxceleb_trainer/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [2]:
%cd ..

/home/iref/Repos/voxceleb_trainer


# Prepare Vox-like test list

In [3]:
frame = pd.read_csv("./data/protocol.csv")

In [4]:
frame

,enroll,verify,enroll_user,verify_user,predict_class
0,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0001.wav,121,121,1
1,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0002.wav,121,121,1
2,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0003.wav,121,121,1
3,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0004.wav,121,121,1
4,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0005.wav,121,121,1
...,...,...,...,...,...
1721026,test-clean/908/31957/908-31957-0022.wav,test-clean/908/31957/908-31957-0024.wav,908,908,1
1721027,test-clean/908/31957/908-31957-0022.wav,test-clean/908/31957/908-31957-0025.wav,908,908,1
1721028,test-clean/908/31957/908-31957-0023.wav,test-clean/908/31957/908-31957-0024.wav,908,908,1
1721029,test-clean/908/31957/908-31957-0023.wav,test-clean/908/31957/908-31957-0025.wav,908,908,1


In [15]:
t = frame.apply(lambda row: str(row["predict_class"]) + " " + row["enroll"] + " " +  row["verify"], axis=1)

In [21]:
with open("./data/ls_test_list.txt", "w+") as f:
    for line in t.to_list():
        f.write(f"{line}\n") 

# Evaluation

In [3]:
ecp = EcapaTrainer(test_path="./data/librispeech/", test_file="./data/ls_test_list.txt", batch_size=256)

Initialised AAMSoftmax margin 0.200 scale 30.000
03-13 16:41:54 Overall parameters: = 6194048.00
Learnable model parameters: 6194048
Learnable loss parameters: 1150848
Sched type: StepLR


In [4]:
ecp.load_params("./exps/Ecapa/sb_augs/11-20/11-20_cp.tar")

20

In [5]:
ecp.eval(20)

Reading 2600 of 2620: 109.08 Hz, embedding size 1
Computing 1721000 of 1721031: 7143.32 Hz
 2023-03-13 16:46:27 Epoch 20, VEER 3.3091, MinDCF 0.16564


(3.3091272934864677, 0.16564214594815027)

In [7]:
hcp = HuCapaTrainer(test_path="./data/librispeech/", test_file="./data/ls_test_list.txt", batch_size=256, max_frames = 200)

Initialised AAMSoftmax margin 0.200 scale 30.000
03-13 16:47:20 Overall parameters: = 118661837.00
Learnable parameters: 24290125
Sched type: StepLR


In [8]:
hcp.load_params("./exps/HuCapa/cyclic_sched_2/16-24/16-24_cp.tar")

24

In [9]:
hcp.eval()

Reading 2600 of 2620: 24.00 Hz, embedding size 192
Computing 1721000 of 1721031: 6726.16 Hz
 2023-03-13 16:53:34 VEER 2.1498, MinDCF 0.11260


(2.149824382075005, 0.11260072423964769)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher = HuCapa(device=device)
student = ECAPA_TDNN(512)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill/0-20_distill_ecapa_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-03-13 16:56:04 [WARNING] Force convert framework model to neural_compressor model.
2023-03-13 16:56:04 [WARNING] Force convert framework model to neural_compressor model.
2023-03-13 16:56:04 [INFO] student_targets_loss: CE, 0.5
2023-03-13 16:56:04 [INFO] teacher_student_loss: MSE, 0.5


PyTorchFXModel(
  (_model): ECAPA_TDNN(
    (torchfbank): Sequential(
      (0): PreEmphasis()
      (1): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
    )
    (conv1): Conv1d(80, 512, kernel_size=(5,), stride=(1,), padding=(2,))
    (relu): ReLU()
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Bottle2neck(
      (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (4): Conv1d(64, 64, kernel_size=(3,), s

In [17]:
def eval(model):
    model.eval()
    sc, lab, _ = evaluateFromList("./data/ls_test_list.txt", "./data/librispeech/", model, 16)
    result = tuneThresholdfromScore(sc, lab, [1, 0.1])
    fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
    mindcf, _ = ComputeMinDcf(fnrs, fprs, thresholds, p_target=0.05, c_miss=1, c_fa=1)
    print('\n',time.strftime("%Y-%m-%d %H:%M:%S"), "Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}".format(20, result[1], mindcf))
    return result[1], mindcf

In [18]:
eval(model)

Reading 2600 of 2620: 123.02 Hz, embedding size 192
Computing 1721000 of 1721031: 6694.94 Hz
 2023-03-13 17:03:08 Epoch 20, VEER 2.6642, MinDCF 0.13470


(2.664172855884561, 0.13469824807435213)

In [3]:
test = Path("./exps/HuCapa/cyclic_sched_2/13-15/")

In [6]:
test.parent.stem

'cyclic_sched_2'